In [1]:
import sys
import os

# Add parent directory (../) to Python's search path
sys.path.append(os.path.abspath(".."))

In [2]:
import numpy as np
import pandas as pd


DATA IMPORT

In [3]:
raw_data = pd.read_excel('/workspaces/GEN-AI-DATA-ANALYST/data/e_commerce.xlsx')

In [4]:
raw_data.head()

,index,Order ID,Cust ID,Gender,Age,Date,Status,Channel,SKU,Category,Size,Qty,currency,Amount,ship-city,ship-postal-code,ship-country,B2B
0,1,171-1029312-3038738,1029312,Women,44,2022-12-04,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,INR,376,MOHALI,140301,IN,False
1,2,405-2183842-2225946,2183842,Women,29,2022-12-04,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,INR,1449,GURUGRAM,122002,IN,False
2,3,171-1641533-8921966,1641533,Women,67,2022-12-04,Delivered,Myntra,SET261-KR-PP-S,Set,S,1,INR,453,KOLKATA,700029,IN,False
3,4,404-7490807-6300351,7490807,Women,20,2022-12-04,Delivered,Amazon,SET110-KR-PP-M,Set,M,1,INR,729,THANJAVUR,613007,IN,False
4,5,403-9293516-4577154,9293516,Women,62,2022-12-04,Delivered,Myntra,JNE2294-KR-A-XXL,kurta,XXL,1,INR,544,GURUGRAM,122001,IN,False


In [ ]:
category_summary = {}

for col in raw_data.columns:
    unique_vals = raw_data[col].dropna().unique()
    num_unique = len(unique_vals)
    category_summary[col] = {
        "Num_Unique": num_unique,
        "Sample_Values": unique_vals[:10]  # show only first 10 for brevity
    }

# Display the results
for col, info in category_summary.items():
    print(f"\nColumn: {col}")
    print(f" - Number of Unique Categories: {info['Num_Unique']}")
    print(f" - Sample Categories: {info['Sample_Values']}")


Column: index
 - Number of Unique Categories: 31047
 - Sample Categories: [ 1  2  3  4  5  6  7  8  9 10]

Column: Order ID
 - Number of Unique Categories: 28471
 - Sample Categories: ['171-1029312-3038738' '405-2183842-2225946' '171-1641533-8921966'
 '404-7490807-6300351' '403-9293516-4577154' '407-1298130-0368305'
 '171-5561216-3398711' '408-2935263-2935550' '404-2648970-9042715'
 '408-0265357-4939534']

Column: Cust ID
 - Number of Unique Categories: 28437
 - Sample Categories: [1029312 2183842 1641533 7490807 9293516 1298130 5561216 2935263 2648970
  265357]

Column: Gender
 - Number of Unique Categories: 4
 - Sample Categories: ['Women' 'Men' 'W' 'M']

Column: Age
 - Number of Unique Categories: 61
 - Sample Categories: [44 29 67 20 62 49 23 70 75 43]

Column: Date
 - Number of Unique Categories: 36
 - Sample Categories: <DatetimeArray>
['2022-12-04 00:00:00', '2022-11-04 00:00:00', '2022-10-04 00:00:00',
 '2022-09-04 00:00:00', '2022-08-04 00:00:00', '2022-07-04 00:00:00',
 '202

2.0 DATA PREPROCESSING

In [6]:
from genai_dataanalyst.assistant import AnalystAssistant
assistant = AnalystAssistant()

2.1 Solves Categorical Problem

If You standardized the Gender column to ensure it contains only 'Men' or 'Women' by replacing abbreviations: 'W' was changed to 'Women' and 'M' to 'Men'. This makes gender data clean and consistent for analysis.

In [7]:
new_data = assistant.clean(raw_data, prompt="Standardize the 'Gender' column so that it only contains 'Men' or 'Women'. Replace 'W' with 'Women' and 'M' with 'Men'.")

[INFO] Received prompt: Standardize the 'Gender' column so that it only contains 'Men' or 'Women'. Replace 'W' with 'Women' and 'M' with 'Men'.
[DEBUG] Generated Code:
 df['Gender'] = df['Gender'].replace({'W': 'Women', 'M': 'Men'})


In [8]:
new_data['Gender'].unique() # New data

array(['Women', 'Men'], dtype=object)

In [9]:
raw_data['Gender'].unique()  # Old data

array(['Women', 'Men', 'W', 'M'], dtype=object)

If You cleaned the Qty column by converting text values like 'One' and 'Two' into their numeric forms 1 and 2, so the entire column now contains only numbers. This makes quantity data consistent and ready for calculations

In [10]:
new_data = assistant.clean(new_data, prompt="Standadize all textual quantity values in the Qty column (e.g., 'One', 'Two') to their numeric equivalents (e.g., 1, 2). Ensure the column is entirely numeric after conversion so datatype should be int ")

[INFO] Received prompt: Standadize all textual quantity values in the Qty column (e.g., 'One', 'Two') to their numeric equivalents (e.g., 1, 2). Ensure the column is entirely numeric after conversion so datatype should be int 


[DEBUG] Generated Code:
 qty_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10}
df['Qty'] = df['Qty'].map(qty_dict).fillna(df['Qty']).astype(int)


<string>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [11]:
new_data['Qty'].unique() # New data


array([1, 2, 4, 3, 5])

In [12]:
raw_data['Qty'].unique()  # Old data

array([1, 'One', 2, 4, 3, 'Two', 5], dtype=object)

In [13]:
new_data.dtypes

index                        int64
Order ID                    object
Cust ID                      int64
Gender                      object
Age                          int64
Date                datetime64[ns]
Status                      object
Channel                     object
SKU                         object
Category                    object
Size                        object
Qty                          int64
currency                    object
Amount                       int64
ship-city                   object
ship-postal-code             int64
ship-country                object
B2B                           bool
dtype: object

3.0 DATA TRANSFORMATION


3.1 Playing With Columns.

I can extract the month and year from the Date column to create two new columns: Order_Month and Order_Year. This makes it easier to analyze sales trends over time, like comparing orders across different months or years.

In [14]:
transformed_data = assistant.clean(new_data, prompt="From the Date column, extract the month and year into two new columns: Order_Month (as full month name) and Order_Year (as 4-digit year).Finally place both the newly created columns after Date column")

[INFO] Received prompt: From the Date column, extract the month and year into two new columns: Order_Month (as full month name) and Order_Year (as 4-digit year).Finally place both the newly created columns after Date column
[DEBUG] Generated Code:
 df['Order_Month'] = df['Date'].dt.strftime('%B')
df['Order_Year'] = df['Date'].dt.strftime('%Y')
cols = df.columns.tolist()
cols.insert(6, cols.pop(cols.index('Order_Month')))
cols.insert(6, cols.pop(cols.index('Order_Year')))
df = df[cols]


In [15]:
transformed_data.head(3)

,index,Order ID,Cust ID,Gender,Age,Date,Order_Year,Order_Month,Status,Channel,SKU,Category,Size,Qty,currency,Amount,ship-city,ship-postal-code,ship-country,B2B
0,1,171-1029312-3038738,1029312,Women,44,2022-12-04,2022,December,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,INR,376,MOHALI,140301,IN,False
1,2,405-2183842-2225946,2183842,Women,29,2022-12-04,2022,December,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,INR,1449,GURUGRAM,122002,IN,False
2,3,171-1641533-8921966,1641533,Women,67,2022-12-04,2022,December,Delivered,Myntra,SET261-KR-PP-S,Set,S,1,INR,453,KOLKATA,700029,IN,False



Suppose i want to create a new column called Age_Group that classifies each person based on their age: anyone under 20 is labeled a Teenager, those between 20 and 49 are labeled Adult, and those 50 or older are labeled Senior. This helps group customers into age based categories for better analysis.

In [16]:
transformed_data = assistant.clean(transformed_data, prompt="Create a new column of 'Age' column named 'Age_Group'.Place the 'Age_Group' column besides 'Age' column  based on the Age column: If Age < 20 → Teenager , If Age ≥ 20 and < 50 → Adult , If Age ≥ 50 → Senior")

[INFO] Received prompt: Create a new column of 'Age' column named 'Age_Group'.Place the 'Age_Group' column besides 'Age' column  based on the Age column: If Age < 20 → Teenager , If Age ≥ 20 and < 50 → Adult , If Age ≥ 50 → Senior
[DEBUG] Generated Code:
 df['Age_Group'] = np.where(df['Age'] < 20, 'Teenager', np.where(df['Age'] < 50, 'Adult', 'Senior'))
df = df[['index', 'Order ID', 'Cust ID', 'Gender', 'Age', 'Age_Group', 'Date', 'Order_Year', 'Order_Month', 'Status', 'Channel ', 'SKU', 'Category', 'Size', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-postal-code', 'ship-country', 'B2B']]


In [17]:
transformed_data.head(3)

,index,Order ID,Cust ID,Gender,Age,Age_Group,Date,Order_Year,Order_Month,Status,...,SKU,Category,Size,Qty,currency,Amount,ship-city,ship-postal-code,ship-country,B2B
0,1,171-1029312-3038738,1029312,Women,44,Adult,2022-12-04,2022,December,Delivered,...,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,INR,376,MOHALI,140301,IN,False
1,2,405-2183842-2225946,2183842,Women,29,Adult,2022-12-04,2022,December,Delivered,...,SET414-KR-NP-L,Set,L,1,INR,1449,GURUGRAM,122002,IN,False
2,3,171-1641533-8921966,1641533,Women,67,Senior,2022-12-04,2022,December,Delivered,...,SET261-KR-PP-S,Set,S,1,INR,453,KOLKATA,700029,IN,False


In [27]:
transformed_data = assistant.clean(transformed_data, prompt="Drop the following columns from the dataset currency, ship-country, index")

[INFO] Received prompt: Drop the following columns from the dataset currency, ship-country, index
[DEBUG] Generated Code:
 df = df.drop(['currency', 'ship-country', 'index'], axis=1)


In [28]:
transformed_data

,Order ID,Cust ID,Gender,Age,Age_Group,Date,Order_Year,Order_Month,Status,Channel,SKU,Category,Size,Qty,Amount,ship-city,ship-postal-code,B2B
0,171-1029312-3038738,1029312,Women,44,Adult,2022-12-04,2022,December,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,376,MOHALI,140301,False
1,405-2183842-2225946,2183842,Women,29,Adult,2022-12-04,2022,December,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,1449,GURUGRAM,122002,False
2,171-1641533-8921966,1641533,Women,67,Senior,2022-12-04,2022,December,Delivered,Myntra,SET261-KR-PP-S,Set,S,1,453,KOLKATA,700029,False
3,404-7490807-6300351,7490807,Women,20,Adult,2022-12-04,2022,December,Delivered,Amazon,SET110-KR-PP-M,Set,M,1,729,THANJAVUR,613007,False
4,403-9293516-4577154,9293516,Women,62,Senior,2022-12-04,2022,December,Delivered,Myntra,JNE2294-KR-A-XXL,kurta,XXL,1,544,GURUGRAM,122001,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31042,405-4724097-1016369,4724097,Men,60,Senior,2022-01-06,2022,January,Delivered,Flipkart,J0083-KR-XL,kurta,XL,1,565,PUNE,412307,False
31043,405-4724097-1016369,4724097,Men,34,Adult,2022-01-06,2022,January,Delivered,Myntra,J0010-LCD-S,Set,S,1,939,BENGALURU,560061,False
31044,408-3572295-5992330,3572295,Men,29,Adult,2022-01-06,2022,January,Delivered,Amazon,SET398-KR-PP-L,Set,L,1,1115,PITHORAGARH,262501,False
31045,405-1423498-9705162,1423498,Men,46,Adult,2022-01-06,2022,January,Delivered,Amazon,JNE3568-KR-M,kurta,M,1,399,DHULE,424006,False


In [29]:
category_summary_transformed = {}

for col in transformed_data.columns:
    unique_vals = transformed_data[col].dropna().unique()
    num_unique = len(unique_vals)
    category_summary_transformed[col] = {
        "Num_Unique": num_unique,
        "Sample_Values": unique_vals[:10]  # show only first 10 for brevity
    }

# Display the results
for col, info in category_summary_transformed.items():
    print(f"\nColumn: {col}")
    print(f" - Number of Unique Categories: {info['Num_Unique']}")
    print(f" - Sample Categories: {info['Sample_Values']}")



Column: Order ID
 - Number of Unique Categories: 28471
 - Sample Categories: ['171-1029312-3038738' '405-2183842-2225946' '171-1641533-8921966'
 '404-7490807-6300351' '403-9293516-4577154' '407-1298130-0368305'
 '171-5561216-3398711' '408-2935263-2935550' '404-2648970-9042715'
 '408-0265357-4939534']

Column: Cust ID
 - Number of Unique Categories: 28437
 - Sample Categories: [1029312 2183842 1641533 7490807 9293516 1298130 5561216 2935263 2648970
  265357]

Column: Gender
 - Number of Unique Categories: 2
 - Sample Categories: ['Women' 'Men']

Column: Age
 - Number of Unique Categories: 61
 - Sample Categories: [44 29 67 20 62 49 23 70 75 43]

Column: Age_Group
 - Number of Unique Categories: 3
 - Sample Categories: ['Adult' 'Senior' 'Teenager']

Column: Date
 - Number of Unique Categories: 36
 - Sample Categories: <DatetimeArray>
['2022-12-04 00:00:00', '2022-11-04 00:00:00', '2022-10-04 00:00:00',
 '2022-09-04 00:00:00', '2022-08-04 00:00:00', '2022-07-04 00:00:00',
 '2022-06-04 00

Complex Transformation Problem

You calculated how much each customer spent in total, then used that to create a new column called CLV_Tier that classifies customers as Low, Medium, High, or VIP. This helps identify your most valuable customers for marketing and sales strategies.

##### Why Complex?
Requires grouping and aggregation by Cust ID.

Involves merging back the tiered result into the original dataset.

Prepares data for segmented marketing, loyalty programs, and revenue prediction models.